In [22]:
FIXED_IP = [2, 6, 3, 1, 4, 8, 5, 7]
FIXED_EP = [4, 1, 2, 3, 2, 3, 4, 1]
FIXED_IP_INVERSE = [4, 1, 3, 5, 7, 2, 8, 6]
FIXED_P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
FIXED_P8 = [6, 3, 7, 4, 8, 5, 10, 9]
FIXED_P4 = [2, 4, 3, 1]

S0 = [[1, 0, 3, 2],
      [3, 2, 1, 0],
      [0, 2, 1, 3],
      [3, 1, 3, 2]]

S1 = [[0, 1, 2, 3],
      [2, 0, 1, 3],
      [3, 0, 1, 0],
      [2, 1, 0, 3]]

KEY = '0111111101'

In [76]:
def permutation(bits,fixed_key):
    new = ''
    for k in fixed_key:
        new+=bits[k-1]
    return new

def left_half(bits):
    return bits[:int(len(bits)/2)]

def right_half(bits):
    return bits[int(len(bits)/2):]

def shift(bits):
    left = left_half(bits)[1:] + left_half(bits)[0]
    right = right_half(bits)[1:] + right_half(bits)[0]
    return left+right

def key1():
    return permutation(shift(permutation(KEY,FIXED_P10)),FIXED_P8)

def key2():
    return permutation(shift(shift(shift(permutation(KEY,FIXED_P10)))),FIXED_P8)

def xor(bits,key):
    new = ''
    for i in range(len(bits)):
        new+= str(int(bits[i]) ^ int(key[i]))
    return new

def lookup_table(bits,table) :
    row = int(bits[0]+bits[3],2)
    col = int(bits[1]+bits[2],2)
    return '{0:02b}'.format(table[row][col])

def fk(bits,key):
    L = left_half(bits)
    R = right_half(bits)
    bits = permutation(R,FIXED_EP)
    bits = xor(bits,key)
    bits = lookup_table(left_half(bits),S0) + lookup_table(right_half(bits),S1)
    bits = permutation(bits,FIXED_P4)
    return xor(bits,L)
    
def encrypt(msg):
    bits = permutation(msg,FIXED_IP)
    temp = fk(bits,key1())
    bits = right_half(bits) + temp
    bits = fk(bits,key2())
    return permutation(bits+temp,FIXED_IP_INVERSE)
    
    
def decrypt(ciphertext):
    bits = permutation(ciphertext,FIXED_IP)
    temp = fk(bits,key2())
    bits = right_half(bits) + temp
    bits = fk(bits,key1())
    return permutation(bits+temp,FIXED_IP_INVERSE)
    

In [77]:
msg = '11101010'
ciphertext = encrypt(msg)

print('Encrypted text is ',ciphertext)

decrypted = decrypt(ciphertext)
print('Decrypted text is ',decrypted)

Encrypted text is  10100010
Decrypted text is  11101010
